In [38]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableLambda, RunnableSequence
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from rec import recommend

In [32]:


load_dotenv()

# Initialize LLM
llm = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct"
)

# Tavily search tool
travily_search_tool = TavilySearch(
    max_results=7,
    topic="general",
    include_images=True
)

In [33]:
travily_search_tool.run("Avengers endgame")

{'query': 'Avengers endgame',
 'follow_up_questions': None,
 'answer': None,
 'images': ['http://hdqwalls.com/download/avengers-endgame-2019-official-new-poster-3p-1280x2120.jpg',
  'http://eclipsemagazine.com/wp-content/uploads/2019/07/Avengers-Endgame.jpg',
  'https://cdn.cdon.com/media-dynamic/images/product/movie/dvd/image1/avengers_endgame_sefi-47496385-.jpg',
  'https://image.tmdb.org/t/p/original/yaRps1bMQLyz54M8ib5YdA2a2RZ.jpg',
  'https://cdn.hmv.com/r/w-3280/HMV/files/08/0801dcb1-b6ea-4564-b253-01f6b1f485ce.637079420210000000.jpg'],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Avengers:_Endgame',
   'title': 'Avengers: Endgame - Wikipedia',
   'content': 'Avengers: Endgame is a 2019 American superhero film based on the Marvel Comics superhero team the Avengers. Produced by Marvel Studios and distributed by Walt',
   'score': 0.89849097,
   'raw_content': None},
  {'url': 'https://www.rottentomatoes.com/m/avengers_endgame',
   'title': 'Avengers: Endgame | Rotten Tomato

In [ ]:
class MovieInfo(BaseModel):
    name: str = Field(description="Name of the movie")
    poster_url: str = Field(description="First image URL found, or 'Not available' if none found")
    description: str = Field(description="Short description of the movie")
    imdb_rating: str = Field(description="IMDb rating (e.g., '8.6/10') or 'Not available'")
    watch_link: str = Field(description="Watch link if available, else 'Not available'")


In [46]:
parser = JsonOutputParser(pydantic_object=MovieInfo)

prompt = PromptTemplate(
    template="""

{format_instructions}

Find the Movie information as given format from the search results:
{tool_output}
""",
    input_variables=["tool_output"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)




In [65]:
chain: RunnableSequence = RunnableLambda(
    lambda x: {"tool_output": travily_search_tool.run(x["query"])}
) | prompt | llm | parser

# Step 5: Run the chain
output = chain.invoke({"query": "Martian"})
print(output)

OutputParserException: Invalid json output: ### Movie Information Extraction

To extract the movie information in the required JSON format, we'll need to process the search results. The required information includes:

* Name of the movie
* Poster URL
* Description
* IMDb rating
* Watch link

#### Code

```python
import json

def extract_movie_info(search_results):
    movie_info = {}

    # Extract the name of the movie
    movie_name = search_results['query'].capitalize()
    movie_info['name'] = movie_name

    # Extract the poster URL
    poster_url = search_results['images'][0] if search_results['images'] else 'Not available'
    movie_info['poster_url'] = poster_url

    # Extract the description
    description = search_results['results'][0]['content'] if search_results['results'] else 'Not available'
    movie_info['description'] = description

    # Extract the IMDb rating
    imdb_url = next((result['url'] for result in search_results['results'] if 'imdb' in result['url']), No

In [60]:
res = recommend("Gravity")
print(res)

['The Right Stuff', 'Moonraker', 'Deep Impact', "The Astronaut's Wife", 'Space Pirate Captain Harlock']


In [63]:
res = recommend("Gravity")
for i in res:
    output = chain.invoke({"query": i})
    print(output)

{'name': 'The Right Stuff', 'poster_url': 'https://facts.net/wp-content/uploads/2023/06/44-facts-about-the-movie-the-right-stuff-1687511855.jpeg', 'description': 'The Right Stuff is a 1983 American epic historical drama film written and directed by Philip Kaufman and based on the 1979 book of the same name by Tom Wolfe.', 'imdb_rating': '7.5/10', 'watch_link': 'Not available'}


OutputParserException: Invalid json output: To generate a JSON instance that conforms to the provided schema, we need to extract the relevant information from the search results. Here's a Python script to accomplish this:

```python
import json

def extract_movie_info(search_results):
    movie_info = {}

    # Find the Wikipedia page for more detailed information
    wikipedia_url = None
    imdb_url = None
    poster_url = None
    watch_link = 'Not available'

    for result in search_results['results']:
        if result['url'] == 'https://en.wikipedia.org/wiki/Moonraker_(film)':
            wikipedia_url = result['url']
        elif result['url'] == 'https://www.imdb.com/title/tt0079574/':
            imdb_url = result['url']

    if wikipedia_url:
        # Fetch the content from Wikipedia (for simplicity, we assume we can get the required info from the provided data)
        wikipedia_content = search_results['results'][0]['content']
        movie_info['name'] = 'Moonraker'
    